In [2]:
from arango import ArangoClient
# pip install python-arango
# Initialize the ArangoDB client.
client = ArangoClient(protocol='http', host='192.168.254.134', port=8529)
# 参考 https://python-driver-for-arangodb.readthedocs.io/en/master/aql.html
# Connect to "test" database as root user.
db = client.db('mydb', username='root', password='root')
cities = db.create_collection('cities')

In [3]:
cities.indexes()
# Add a new hash index on document fields "continent" and "country".
index = cities.add_hash_index(fields=['continent', 'country'], unique=True)

# Add new fulltext indexes on fields "continent" and "country".
index = cities.add_fulltext_index(fields=['continent'])
index = cities.add_fulltext_index(fields=['country'])

# Add a new skiplist index on field 'population'.
index = cities.add_skiplist_index(fields=['population'], sparse=False)

# Add a new geo-spatial index on field 'coordinates'.
index = cities.add_geo_index(fields=['coordinates'])

# Add a new persistent index on fields 'currency'.
index = cities.add_persistent_index(fields=['currency'], sparse=True)

# Delete the last index from the collection.
cities.delete_index(index['id'])

[{'fields': ['_key'],
  'id': '0',
  'sparse': False,
  'type': 'primary',
  'unique': True,
  'selectivity': 1}]

In [4]:
db.collection('cities').insert_many([
    {'_key': 'Abby', 'age': 22},
    {'_key': 'John', 'age': 18},
    {'_key': 'Mary', 'age': 21}
])

[{'_id': 'cities/Abby', '_key': 'Abby', '_rev': '_ZFWhh2S--_'},
 {'_id': 'cities/John', '_key': 'John', '_rev': '_ZFWhh2W--_'},
 {'_id': 'cities/Mary', '_key': 'Mary', '_rev': '_ZFWhh2W--B'}]

In [6]:
# Get the AQL API wrapper.
aql = db.aql

# Retrieve the execution plan without running the query.
aql.explain('FOR doc IN cities RETURN doc')

# Validate the query without executing it.
aql.validate('FOR doc IN cities RETURN doc')

{'parsed': True,
 'collections': ['cities'],
 'ast': [{'type': 'root',
   'subNodes': [{'type': 'for',
     'subNodes': [{'type': 'variable', 'name': 'doc', 'id': 0},
      {'type': 'collection', 'name': 'cities'}]},
    {'type': 'return',
     'subNodes': [{'type': 'reference', 'name': 'doc', 'id': 0}]}]}],
 'bind_vars': []}

In [10]:
# Execute the query
cursor = db.aql.execute(
  'FOR doc IN cities FILTER doc.age < @value RETURN doc',
  bind_vars={'value': 19}
)
# Iterate through the result cursor
student_keys = [doc['_key'] for doc in cursor]

student_keys

['John']